# **Segmenting and Clustering Neighborhoods in Toronto : part 3 (from line 20).**
## **Applied Data Science Capstone Week 3 Peer-Graded Assignment.**

### **Let us install some of the libraries which we will need later on.**

In [1]:
import sys
!{sys.executable} -m pip install lxml
print("lxml installed")
!{sys.executable} -m pip install beautifulsoup4
print("beautifulsoup4 installed")
!{sys.executable} -m pip install geopy
print("geopy installed")
!{sys.executable} -m pip install folium
print("folium installed")

lxml installed
beautifulsoup4 installed
geopy installed
folium installed


### **Importing required libraries.**

In [2]:
import lxml
import lxml.html as lh

import requests # library to handle requests
from requests import get
import urllib.request

from bs4 import BeautifulSoup # for webscraping import Beautiful Soup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import csv



### **Scraping the wikipedia page.**

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"   # link of the destination wikipedia page.
page  = urllib.request.urlopen(link).read()   # source code of the destination page

In [4]:
# step by step scraping of the above source code of the destination page to get the table data we require

# parsing the data in the variable 'page' by using lxml
full_page = BeautifulSoup(page, 'lxml')

# finding the required table 
table_data = full_page.find('table', class_ = 'wikitable')

# finding the required data in the above identified table
trs = table_data.find_all('tr')

### **Now let us move forward with making a pandas dataframe with these data.**
#### **We will do that by first aggregating these data into a variable.** 

In [5]:
data = ""                                # its an empty string in which we will aggregate our whole data.
for x in trs:                            # 'trs' is the variable in which the data is stored from web-scrapping the wikipedia page.
    row = ""                             # its another empty string in which we will aggregate row wise data.
    for y in x.find_all('td'): 
        row = row + "," + y.text         # using ',' as a delimiter we are aggregating the data according to each row. 
    data = data + row                    # then we are appending the variable 'data' by adding each row to it.
print(data)

,M1A,Not assigned,Not assigned
,M2A,Not assigned,Not assigned
,M3A,North York,Parkwoods
,M4A,North York,Victoria Village
,M5A,Downtown Toronto,Harbourfront
,M6A,North York,Lawrence Heights
,M6A,North York,Lawrence Manor
,M7A,Downtown Toronto,Queen's Park
,M8A,Not assigned,Not assigned
,M9A,Queen's Park,Not assigned
,M1B,Scarborough,Rouge
,M1B,Scarborough,Malvern
,M2B,Not assigned,Not assigned
,M3B,North York,Don Mills North
,M4B,East York,Woodbine Gardens
,M4B,East York,Parkview Hill
,M5B,Downtown Toronto,Ryerson
,M5B,Downtown Toronto,Garden District
,M6B,North York,Glencairn
,M7B,Not assigned,Not assigned
,M8B,Not assigned,Not assigned
,M9B,Etobicoke,Cloverdale
,M9B,Etobicoke,Islington
,M9B,Etobicoke,Martin Grove
,M9B,Etobicoke,Princess Gardens
,M9B,Etobicoke,West Deane Park
,M1C,Scarborough,Highland Creek
,M1C,Scarborough,Rouge Hill
,M1C,Scarborough,Port Union
,M2C,Not assigned,Not assigned
,M3C,North York,Flemingdon Park
,M3C,North York,Don Mills South
,M4C,East York,Woodbine Height

#### **Then making a CSV file and writing the data in that CSV file.**

In [6]:
data_file = open("toronto_data.csv",'w')   
data_file.write(data)
data_file.close()

# so we have a variable now called 'data_file' with all the data in it and ',' as a delimiter.

#### **And lastly converting that into a pandas dataframe.**

In [7]:
data_file_df = pd.read_csv('toronto_data.csv',header= None)            # converting the CSV fie into a pandas dataframe

data_file_df.head()

,0,1,2,3
0,NaN,M1A,Not assigned,Not assigned
1,NaN,M2A,Not assigned,Not assigned
2,NaN,M3A,North York,Parkwoods
3,NaN,M4A,North York,Victoria Village
4,NaN,M5A,Downtown Toronto,Harbourfront


#### **Okay so now let's clean our dataset.**

#### **The first step towards cleaning the dataset is making sure that we have only three columns here : 'PostalCode', 'Borough', 'Neighborhood'.**
#### **So we have to delete the first column in our dataset.**
#### **And rename the columns.**

In [8]:
data_file_df.drop(columns = [0], axis= 1, inplace = True)   # droping the first column in the dataframe as it has only NaN values which are not required
data_file_df.head()

,1,2,3
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
data_file_df.columns = ['PostalCode', 'Borough', 'Neighborhood']       # naming the columns
data_file_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


####  **Now that we have our dataset fairly clean; the second step is to pre-process our dataset.**
#### **In order to do that we have to do three things :**
##### **1. Only process the cells which have assigned boroughs, ignoring the rest of the cells with a borough that is 'Not assigned'.**
##### **2. Grouping the dataset by postal codes, because more that one neighborhood can exist in a postal code.**
##### **3. If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the **same** as the borough.**

In [10]:
data_file_df.drop(data_file_df.loc[data_file_df['Borough']== 'Not assigned'].index, inplace=True)  # Dropping cells with a Borough that is 'Not assigned'.
data_file_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [11]:
data_file_df.loc[data_file_df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = data_file_df['Borough']   # asigning the neighborhood = borough for a not assigned neighborhood
data_file_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [12]:
data_file_df = data_file_df.groupby(['PostalCode',], sort= False,).agg( ', '.join)   # grouping the dataset by the postal code
data_file_df.head()

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,"North York, North York","Lawrence Heights, Lawrence Manor"
M7A,Downtown Toronto,Queen's Park


In [13]:
data_file_df.reset_index(inplace = True) # resetting the index of the dataset to 0 initial.
data_file_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,"North York, North York","Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [14]:
data_file_df.shape

(103, 3)

#### **Nest step is we need to get the latitude and the longitude coordinates of each neighborhood; in order to utilize the Foursquare location data.**
##### **For that we will be using the given link in the course submission page in order to get the latitude and longitude data that is required.**

In [15]:
lat_long_data_url='http://cocl.us/Geospatial_data' # our destination URL consisting of the CSV file of our data.

# sending a HTTP request, opening and saving the contents of the response as 'tf', and opening a csv file to write the contens of the response to it.
with urllib.request.urlopen(lat_long_data_url) as tf, open('lat_long_data.csv', 'w') as f:       
    f.write(tf.read().decode())                                                             # writing the contents of the response to the opened csv file.
    f.close()                                                                               # finally closing the csv file.
    

In [16]:
df_lat_long = pd.read_csv('lat_long_data.csv')
df_lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
# renming the coluomn name from 'Postal Code' to 'PostalCode' ; so that it matches the column name from the 'data_file_df'.
df_lat_long = df_lat_long.rename(columns = {"Postal Code" : "PostalCode"})
# df_lat_long["Postal Code"] = "PostalCode"
df_lat_long.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### **Okay now we need to merge the two datasets.**

In [18]:
final_data = pd.merge(data_file_df, df_lat_long, on='PostalCode')
final_data.head(25)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,"North York, North York","Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,"Scarborough, Scarborough","Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,"East York, East York","Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,"Downtown Toronto, Downtown Toronto","Ryerson, Garden District",43.657162,-79.378937


In [19]:
final_data.shape

(103, 5)

#### **Okay so next we will explore and cluster the neighborhoods in Toronto.**

##### **Using the  'geopy' library let us get the latitude and longitude values of Toronto city.**

In [20]:
# we already have geopy installed, lets import 'Nominatim' from geopy.
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

##### **In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent 'toronto_explorer', as shown below.**

In [21]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


##### **Okay so now that we have the co-ordinates of 'Toronto City', let us create a map of the same with neighborhoods superimposed on top.**

In [22]:
# we have already installed 'folium' which is a map rendering library.
# lets us import the library now to work with it.
import folium


In [23]:
# create map of Toronto City using the latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_data['Latitude'], final_data['Longitude'], final_data['Borough'], final_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### **Okay now let us simplify the above map and segment and cluster only the neighborhoods in boroughs containing the word 'Toronto'.** 
##### **So let's slice the original dataframe and create a new dataframe of the above stated.**

In [24]:
toronto_data=final_data[final_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,M5B,"Downtown Toronto, Downtown Toronto","Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [25]:
toronto_data = toronto_data.reset_index(drop = True) # resetting the index to 0 initial.
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,"Downtown Toronto, Downtown Toronto","Ryerson, Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


##### **okay so now Let us get the geographical coordinates of Toronto, which we already did using the geocoder.**

In [26]:
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


##### **As we did with all of Toronto City, let's visualize the neighborhoods which are belonging to the boroughs containing the word 'Toronto.**

In [27]:
# create map of Manhattan using latitude and longitude values
map_toronto_data = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=1,
        parse_html=False).add_to(map_toronto_data)  
    
map_toronto_data

#### **Okay let us start utilizing the Foursquare API to explore the neighborhoods and segment them.**

##### **Defining Foursquare Credentials and Version.**

##### **Let's explore the first neighborhood in our dataframe.**
##### **Get the neighborhood's name.**

In [28]:
toronto_data.loc[0, 'Neighborhood']

'Harbourfront'

##### **Get the neighborhood's latitude and longitude values.**

In [29]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


##### **Now, let's get the top 100 venues that are in harbourfront within a radius of 500 meters.**

##### **First, let's create the GET request URL. Name your URL url.**

In [30]:
radius = 500 # defining radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
CLIENT_ID = 'WF51JHXY43ZZIXWLTYBZXJOF1IKY5K3AKDWIJHYU503OU0FQ'
CLIENT_SECRET = 'MUCECBY3FNHAK3HMITAPCT0XUMS4MLQYKD2NFL2YKAL4OCJV'
VERSION = '20190605'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=WF51JHXY43ZZIXWLTYBZXJOF1IKY5K3AKDWIJHYU503OU0FQ&client_secret=MUCECBY3FNHAK3HMITAPCT0XUMS4MLQYKD2NFL2YKAL4OCJV&v=20190605&ll=43.6542599,-79.3606359&radius=500&limit=100'

##### **Send the GET request and examine the resutls.**

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2a841b9388d7001bdb6d8f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

##### **From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.**

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

##### **Now we are ready to clean the json and structure it into a pandas dataframe.**

In [33]:
from pandas.io.json import json_normalize # importing json_normalise that tranforms JSON file into a pandas dataframe

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


##### **And how many venues were returned by Foursquare?**

In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


#### **Explore Neighborhoods in boroughs containing the word 'Toronto'.**

##### **Let's create a function to repeat the same process to all the neighborhoods in boroughs containing the word 'Toronto'.**

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### **Now write the code to run the above function on all neighborhoods in boroughs containing the word 'Toronto' and create a new dataframe called toronto_borough_venues.**

In [37]:
toronto_borough_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

##### **Let's check the size of the resulting dataframe.**

In [38]:
print(toronto_borough_venues.shape)
toronto_borough_venues.head()

(1720, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


##### **Let's check how many venues were returned for each neighborhood.**

In [39]:
toronto_borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,14,14,14,14,14,14
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",86,86,86,86,86,86
Christie,18,18,18,18,18,18


##### **Let's find out how many unique categories can be curated from all the returned venues.**

In [40]:
print('There are {} uniques categories.'.format(len(toronto_borough_venues['Venue Category'].unique())))

There are 232 uniques categories.


#### **Analyze Each Neighborhood .**

In [41]:
# one hot encoding
toronto_in_borough_onehot = pd.get_dummies(toronto_borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_in_borough_onehot['Neighborhood'] = toronto_borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_in_borough_onehot.columns[-1]] + list(toronto_in_borough_onehot.columns[:-1])
toronto_in_borough_onehot = toronto_in_borough_onehot[fixed_columns]

toronto_in_borough_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

##### **And let's examine the new dataframe size.**

In [42]:
toronto_in_borough_onehot.shape

(1720, 232)

##### **Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.**

In [43]:
toronto_in_borough_grouped = toronto_in_borough_onehot.groupby('Neighborhood').mean().reset_index()
toronto_in_borough_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.00,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.070000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.00000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

##### **Let's confirm the new size**.

In [44]:
toronto_in_borough_grouped.shape

(39, 232)

##### **Let's print each neighborhood along with the top 5 most common venues.**

In [45]:
num_top_venues = 5

for hood in toronto_in_borough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_in_borough_grouped[toronto_in_borough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3  Seafood Restaurant  0.03
4      Farmers Market  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0     Coffee Shop  0.10
1  Breakfast Spot  0.10
2            Café  0.10
3   Grocery Store  0.05
4   Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Yoga Studio  0.07
1       Auto Workshop  0.07
2                 Spa  0.07
3          Skate Park  0.07
4  Light Rail Station  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Ai

##### **Let's put that into a pandas dataframe.**

##### **First, let's write a function to sort the venues in descending order.**

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### **Now let's create the new dataframe and display the top 10 venues for each neighborhood.**

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_in_borough_grouped['Neighborhood']

for ind in np.arange(toronto_in_borough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_in_borough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Bar,Café,Hotel,Cosmetics Shop,Restaurant,Thai Restaurant,Asian Restaurant,Concert Hall
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Steakhouse,Beer Bar,Seafood Restaurant,Café,Cheese Shop,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Smoke Shop,Light Rail Station,Spa,Farmers Market,Fast Food Restaurant,Brewery,Burrito Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Boutique


#### **Cluster Neighborhoods.**

#### **Let us import k-means from sklearn.**

In [48]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

##### **Run k-means to cluster the neighborhood into 5 clusters.**

In [49]:
# set number of clusters
kclusters = 5

toronto_in_borough_grouped_clustering = toronto_in_borough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_in_borough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

##### **Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.**

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Farmers Market,Event Space
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Gym,Park,College Auditorium,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Restaurant,Café
2,M5B,"Downtown Toronto, Downtown Toronto","Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Fast Food Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Plaza
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Cocktail Bar,Clothing Store,Hotel,Bakery,Beer Bar,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Women's Store,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


##### **Finally, let's visualize the resulting clusters.**

##### **Let us import the needed libraries for the visualizations.**

In [51]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### **Examine Clusters.**

##### **Now, let us examine each cluster and determine the discriminating venue categories that distinguish each cluster.**

##### **Cluster 1.**

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Farmers Market,Event Space
1,Downtown Toronto,0,Coffee Shop,Gym,Park,College Auditorium,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Restaurant,Café
2,"Downtown Toronto, Downtown Toronto",0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Fast Food Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Plaza
3,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Cocktail Bar,Clothing Store,Hotel,Bakery,Beer Bar,Breakfast Spot
4,East Toronto,0,Trail,Health Food Store,Pub,Women's Store,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Steakhouse,Beer Bar,Seafood Restaurant,Café,Cheese Shop,Breakfast Spot
6,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Juice Bar,Burger Joint,Ice Cream Shop,Chinese Restaurant,Salad Place
7,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Baby Store,Diner,Athletics & Sports,Italian Restaurant,Candy Store,Coffee Shop
8,"Downtown Toronto, Downtown Toronto, Downtown T...",0,Coffee Shop,Steakhouse,Bar,Café,Hotel,Cosmetics Shop,Restaurant,Thai Restaurant,Asian Restaurant,Concert Hall
9,"West Toronto, West Toronto",0,Bakery,Pharmacy,Grocery Store,Park,Music Venue,Middle Eastern Restaurant,Café,Brewery,Bar,Bank


##### **Cluster 2.**

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Garden,Women's Store,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


##### **Cluster 3.**

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,2,Hotel,Department Store,Sandwich Place,Park,Gym,Breakfast Spot,Convenience Store,Food & Drink Shop,Discount Store,Dog Run


##### **Cluster 4.**

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Lawyer,Swim School,Bus Line,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
21,"Central Toronto, Central Toronto",3,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Women's Store,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


##### **Cluster 5.**

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Central Toronto, Central Toronto",4,Park,Playground,Trail,Summer Camp,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Downtown Toronto,4,Park,Playground,Trail,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
